# Workspace

In [ ]:
import os
import requests
import json
from IPython.display import JSON
from IPython.display import Javascript
from IPython.display import clear_output
from IPython.display import display
from IPython.display import Markdown

import sys
sys.path.append('../')
from modules.helpers import get_access_token, load_eoepca_state, test_cell, test_results

## Initialise the environment

In [ ]:
load_eoepca_state()

token_endpoint = (
    f"{os.environ.get('HTTP_SCHEME')}://auth.{os.environ.get('INGRESS_HOST')}"
    f"/realms/{os.environ.get('REALM')}/protocol/openid-connect/token"
)
print(f"Token endpoint: {token_endpoint}")

workspace_api_endpoint = f"{os.environ.get('HTTP_SCHEME')}://workspace-api.{os.environ.get('INGRESS_HOST')}"
print(f"Workspace API endpoint: {workspace_api_endpoint}")

## Helper for authenticated Workspace sessions

In [ ]:
workspace_api_client_id = os.environ.get("WORKSPACE_API_CLIENT_ID")
workspace_api_client_secret = os.environ.get("WORKSPACE_API_CLIENT_SECRET")
sessions = {}
def get_session(username, password):
    global sessions
    if not username in sessions:
        sessions[username] = requests.Session()
    token = get_access_token(username, password, workspace_api_client_id, workspace_api_client_secret)
    if token:
        sessions[username].headers.update({"Authorization": f"Bearer {token}"})
    return sessions[username]

## Workspace Details

As an 'admin' user, we can lookup the details for the Workspace of the _Test User_ created during deployment.

This is acheived via a `GET` request to the Workspace API `/workspaces/<workspace-name>` endpoint.

In [ ]:
admin_user = os.environ.get("KEYCLOAK_TEST_ADMIN")
admin_password = os.environ.get("KEYCLOAK_TEST_PASSWORD")
test_user = os.environ.get("KEYCLOAK_TEST_USER")

response = get_session(admin_user, admin_password).get(f"{workspace_api_endpoint}/workspaces/ws-{test_user}")
print(f"Details of Workspace for test user '{test_user}'")
JSON(response.json(), expanded=True)

## New Workspace Creation

As an 'admin' user, we can create new Workspaces.

In this case we will first create a new user `frank`, and then create `frank` a corresponding workspace.<br>
Workspaces are prefixed `ws-` - which results in the full workspace name `ws-frank`.

**Create user `frank`**

In [ ]:
%%bash

# Specify Frank's password via a secret
kubectl -n iam-management create secret generic frank-password --from-literal=password=${KEYCLOAK_TEST_PASSWORD}

# Create the user frank via the Crossplane Keycloak Provider using a `User` CRD
kubectl apply -f user-frank.yaml

**Create workspace for user `frank`**

Workspace creation is achieved via a `POST` request to the Worksapce API `/workspaces` endpoint.

In [ ]:
payload = {
    "preferred_name": "frank",
    "default_owner": "frank"
}
response = get_session(admin_user, admin_password).post(f"{workspace_api_endpoint}/workspaces", json=payload)
print(f"Workspace creation response: {response.text}")

## Check Workspace Creation

The Workspace for Frank is created via a pipeline that performs the full setup, including:
* Creation of a dedicated Kubernetes namespace
* Creation of a dedicated object storage bucket via a `Storage` CRD
* Creation of a dedicated interactive environment via a `Datalab` CRD

**Namespace**

In [ ]:
!kubectl get ns ws-frank

**Custom Resources**

Each Workspace is created declaratively via Crossplane Custom Resources - specifically Kubernetes CRDs for `Storage` and `Datalab`.

In [ ]:
!kubectl -n workspace get storage,datalab

## Check Workspace details

Authenticate as user `frank` and get the details for the new workspace.

In [ ]:
frank_password = os.environ.get("KEYCLOAK_TEST_PASSWORD")

response = get_session("frank", frank_password).get(f"{workspace_api_endpoint}/workspaces/ws-frank")
print(f"Details of Workspace for user 'frank'")
JSON(response.json(), expanded=True)

## Open the UI for Frank's Workspace

Authenticate as user Frank to access the workspace via its dedicated URL.

In [ ]:
url = f"{os.environ.get('HTTP_SCHEME')}://workspace-api.{os.environ.get('INGRESS_HOST')}/workspaces/ws-frank"

# Print login details
display(Markdown(f"""
**Login details for Frank:**
* USERNAME: frank
* PASSWORD: {os.environ.get("KEYCLOAK_TEST_PASSWORD")}
"""
))
display(Markdown("**Hit ENTER to open Frank's workspace UI in browser...**"))
input()
clear_output()

# Open workspace in browser
print(f"URL for Frank's workspace: {url}")
Javascript(f'window.open("{url}");')

## Explore the Workspace UI

Within Frank's workspace UI explore via the following actions.

**Dashboard**

View storage credentials in the dashboard.

**Datalab**

Select `Datalab` to start the session (may take a short time to initialise).

Frank's environment is pre-configured to seamless access to:
* dedicated vCluster Kubernetes instance
* pre-provisioned S3 object storage bucket

This environment applies in both the `Terminal` and the `Editor` tooling.

**Terminal**

<span style="color: #777; font-weight: bold; font-style: italic;">
Check access to Frank's vCluster...
</span>

```
kubectl get all -A
```

<span style="color: #777; font-weight: bold; font-style: italic;">
Verify S3 connectivity...
</span>

```
aws s3 ls
aws s3 ls s3://ws-frank --recursive
```

**Editor**

Check the VS Code style IDE.

Use keystroke <kbd>Ctrl-`</kbd> (backtick) to open the IDE in-built terminal.

Repeat the `Terminal` commands here.

**Data**

Browse the file contents of the object storage bucket.

## Access Frank's vCluster

Within the workspace Frank has a vCluster that provides a sandboxed Kubernetes cluster.

To interact with this vCluster we need to take the following steps:
* establish a port-forward connection to the Kubernetes API server of the vCluster
* extract the kubeconfig from a secret within the cluster
* update the `server:` URL in the kubeconfig for the `localhost` port forward connection

**Setup**

In [ ]:
%%bash

# Port forward to the vCLuster API server (background)
pkill -f "kubectl -n ws-frank-default-vc port-forward svc/my-vcluster 8443:https"  # clean start
nohup kubectl -n ws-frank-default-vc port-forward svc/my-vcluster 8443:https >/tmp/pf.log 2>&1 &

# Extract the kubeconfig from the secret
kubectl -n ws-frank-default-vc get secret ws-frank-default-vc-kubeconfig -o jsonpath='{.data.config}' | base64 -d > frank-kubeconfig.yaml

# Update kubeconfig for the localhost port forward
sed -i 's|^\([[:space:]]*\)server:[[:space:]].*$|\1server: https://localhost:8443|' frank-kubeconfig.yaml

**Deploy Test Workload**

Deploy a test nginx workload within the vCluster

In [ ]:
%%bash

export KUBECONFIG=frank-kubeconfig.yaml

# Deploy
kubectl apply -f nginx-test.yaml
kubectl rollout status deployment/nginx-test

# Check
kubectl get svc,deploy,pods -l app=nginx-test

**Expose the test service via Workspace IDE**

Use the Workspace VS Code IDE to expose the service via a port-forward.

In the VS Code Editor open a terminal with the <kbd>Ctrl-`</kbd> (backtick) key sequence.

Initiate port forwarding to the nginx test service...

```
kubectl port-forward svc/nginx-test 5000:80
```

The VS Code IDE recognises the local service on port `5000` and automatically exposes through the endpoint `https://editor-ws-frank-default.<ingress-host>/proxy/5000/`.

**Open the exposed endpoint in your browser**

In [ ]:
url = f"{os.environ.get('HTTP_SCHEME')}://editor-ws-frank-default.{os.environ.get('INGRESS_HOST')}/proxy/5000/"
print(f"URL for exposed nginx test service: {url}")
Javascript(f'window.open("{url}");')

**Teardown**

Quit the port forward to the nginx test service in the Workspace IDE VS Code terminal - using <kbd>Ctrl-C</kbd>.

**Kill port-forward to vCluster**

In [ ]:
%%bash

# Kill the port forward
pkill -f "kubectl -n ws-frank-default-vc port-forward svc/my-vcluster 8443:https"

# Remove Frank's kubeconfig file
rm -f frank-kubeconfig.yaml

## Delete Workspace

As an 'admin' user, we can delete an existing Workspace, such as Frank's created during this notebook.

This is acheived via a DELETE request to the Workspace API /workspaces/<workspace-name> endpoint.

In [ ]:
response = get_session(admin_user, admin_password).delete(f"{workspace_api_endpoint}/workspaces/ws-frank")
print(f"Workspace deletion response code: {response.status_code}")

## Delete 'Frank' Test User

In [ ]:
%%bash
kubectl delete -f user-frank.yaml
kubectl -n iam-management delete secret frank-password